# Install Libraries

In [1]:
!pip install -r /kaggle/input/requirements-file/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 33.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 M

In [48]:
!pip install -q gradio

# Import Libraries

In [49]:
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from kaggle_secrets import UserSecretsClient
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

import torch
import pandas as pd
import os
import re
import gradio as gr

# Load Data

In [3]:
try:
    df = pd.read_csv('/kaggle/input/tmdb-movies-processed/processed_2024_tmdb_movies.csv')
    df['page_content'] = df['page_content'].fillna("")
    print(f"Successfully loaded {len(df)} movies.")
except FileNotFoundError:
    print("Error: Could not find 'processed_movies.csv'.")

Successfully loaded 26075 movies.


# Load Embedding Model

In [4]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
print("Embedding Model Loaded")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding Model Loaded


# Prepare Document

In [5]:
documents = []

In [6]:
for i, r in df.iterrows():
    doc = Document(
        page_content=r['page_content'],
        metadata={
            'id': r['id'],
            'title': r['title'],
            'rating': r['vote_average'],
            'year': r['year'],
            'genres': r['genres']
        }
    )
    documents.append(doc)

# Create Database

In [7]:
print(f'Ingesting {len(documents)} movies into ChromaDB...')

vector_db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory='./chroma_db_data',
)
print("✅ Success! Database created.")

Ingesting 26075 movies into ChromaDB...
✅ Success! Database created.


# Test

In [8]:
query = 'A movie about a psychological mind game'
print(f"\nTest Query: '{query}'")

results = vector_db.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\nResult {i+1}: {doc.metadata['title']} (Rating: {doc.metadata['rating']})")
    snippest = doc.page_content.replace('\n', ' ').strip()[:150]
    print(f'Context: {snippest}...')


Test Query: 'A movie about a psychological mind game'

Result 1: Elephant Song (Rating: 7.09)
Context: Movie Title: Elephant Song     Genres: Drama, Mystery     Keywords: mental institution, mind game     Rating: 7.09     Overview: A psychiatrist is dra...

Result 2: Gamer (Rating: 5.7)
Context: Movie Title: Gamer     Genres: Action, Thriller, Science Fiction     Keywords: mind control, prisoner, video game, dystopia, gun battle, wrongful impr...

Result 3: Brainscan (Rating: 6.207)
Context: Movie Title: Brainscan     Genres: Horror     Keywords: dream, video game, virtual reality, pretended murder, hypnosis, trance     Rating: 6.207     O...


# Chat Bot

## Setup Model

In [9]:
model_id = "HuggingFaceH4/zephyr-7b-beta"

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00005-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [12]:
# Create the pipeline
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=512,
)

Device set to use cuda:0


In [13]:
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [14]:
print('setup model done successfully')

setup model done successfully


## Smart Retriever

In [23]:
def get_smart_retriever(query, db):
    # Default: fetch 10, return top 4 (MMR for diversity)
    search_kwargs = {"k": 4, "fetch_k": 10, "lambda_mult": 0.25}
    filter_dict = {}
    
    # Check for "after YYYY"
    year_match = re.search(r'after (\d{4})', query.lower())
    if year_match:
        year = int(year_match.group(1))
        filter_dict["year"] = {"$gt": year}
        print(f"   🔎 Filter Active: Movies released after {year}")
    
    # Check for specific genres (Simple keyword check)
    genres = ["action", "comedy", "horror", "drama", "scifi", "romance"]
    for g in genres:
        if g in query.lower():
            # Note: This assumes your CSV genre format matches (e.g., "Action")
            # We use $contains logic if available, or just rely on vector search + year filter
            pass 

    if filter_dict:
        search_kwargs["filter"] = filter_dict
        
    return db.as_retriever(search_type="mmr", search_kwargs=search_kwargs)

## Prompt

In [43]:
prompt_template = """
<|system|>
You are a precision movie expert. 
Your goal is to recommend movies based strictly on the provided Context.

Guidelines:
1. ALWAYS list the Name and Year of the movie explicitly, e.g., "Movie Name (2023)".
2. Do NOT hallucinate. If the Context says "Year: 2023", do not say it was released before 2022.
3. Be direct and concise. Do not apologize. Do not explain your database limitations.
4. If multiple movies match, list them and mention which has the highest rating.

Context:
{context}
</s>
<|user|>
{question}
</s>
<|assistant|>
"""

In [44]:
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=['context', 'question']
)

## Run Bot

In [45]:
def chat_with_bot(user_query):
    # 1. Get the correct retriever (with filters if needed)
    smart_retriever = get_smart_retriever(user_query, vector_db)
    
    # 2. Build the chain dynamically
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=smart_retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    
    # 3. Run
    print(f"\nUser: {user_query}")
    print("Thinking...")
    response = qa_chain.invoke(user_query)
    print(f"Bot: {response['result']}")

In [46]:
chat_with_bot("Recommend a movie about the atomic bomb released after 2022")

   🔎 Filter Active: Movies released after 2022

User: Recommend a movie about the atomic bomb released after 2022
Thinking...
Bot: I'm sorry but based on the context provided, there is no movie about the atomic bomb released after 2022. The given context only mentions "Oppenheimer" as a movie about the atomic bomb, and according to the context, it was released in 2023. If you provide more context or additional information, I may be able to recommend a different movie. However, as of now, my recommendation would be "Oppenheimer" (2023).


In [47]:
chat_with_bot("Recommend an action movie released after 2026")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


   🔎 Filter Active: Movies released after 2026

User: Recommend an action movie released after 2026
Thinking...
Bot: I'm sorry but as an AI language model, I don't have access to future releases or information about upcoming movies. Therefore, I'm unable to provide you with a specific title for an action movie released after 2026. However, you can check out some upcoming action movies that have been announced by production studios and keep an eye out for their release dates. Some of these movies include "The Witcher: Blood Origin" (2022), "Shazam! Fury of the Gods" (2023), "John Wick: Chapter 4" (2023), "Black Panther: Wakanda Forever" (2022), and "Avatar: The Way of Water" (2022). Amongst these, "John Wick: Chapter 4" has received high ratings from critics and fans alike, making it one of the most anticipated action movies set to be released in the near future.


# Gradio UI

In [50]:
def chat_interface(message, history):  
    smart_retriever = get_smart_retriever(message, vector_db)
    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=smart_retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    
    response = qa_chain.invoke(message)
    return response['result']

In [51]:
demo = gr.ChatInterface(
    fn=chat_interface,
    title="🎬 Cine-RAG: The AI Movie Expert",
    description="Ask me about movies! I know everything up to 2024. Try 'Action movies after 2022' or 'Movies about AI'.",
    theme="soft",
    examples=["Recommend a sci-fi movie about AI", "Action movies after 2022", "Horror movies from the 80s"],
)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


In [52]:
print("🚀 Launching UI...")
demo.launch(share=True, debug=True)

🚀 Launching UI...
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://36f8775e52b06fb028.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


   🔎 Filter Active: Movies released after 2022
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://36f8775e52b06fb028.gradio.live
